In [2]:
# !pip freeze > requirements.txt


In [71]:
!pip install dash_bootstrap_components

  Using cached dash_bootstrap_components-1.4.1-py3-none-any.whl (220 kB)


In [3]:
import pandas as pd
import plotly.graph_objects as go

In [43]:
df_og = pd.read_csv("data/jobs.csv")

# adding a column to ID companies that ghosted after interview
df_og['Ghosted after interview'] = 0
ghosters = df_og.loc[df_og['Company']=='OrderEase'].index[0]
df_og.at[ghosters, 'Ghosted after interview'] = 1
df_og

,Title,Company,Created at,List,Source,Ghosted after interview
0,"Principal Data Scientist , Search and Personal...",Getty Images,"May 19 2023, 09:00 am",apply to,linkedin.com,0
1,Trading Application Support Analyst,John Wiley & Sons,"May 19 2023, 08:22 am",apply to,linkedin.com,0
2,Computational Cancer Biologist,Vevo Therapeutics,"May 19 2023, 07:00 am",applied,linkedin.com,0
3,Data Engineer,Aja Hannah,"May 19 2023, 07:00 am",applied,linkedin.com,0
4,Data Analytics Engineer/Specialist,ConocoPhillips,"May 19 2023, 06:39 am",applied,indeed.com,0
...,...,...,...,...,...,...
139,Product Support Analyst,Central Michigan University,"Apr 17 2023, 06:05 pm",applied,indeed.com,0
140,Performance Analyst – Hydroelectric energy,Innergex,"Apr 17 2023, 06:05 pm",applied,indeed.com,0
141,"Data Analyst, Business Optimisation",VOSKER,"Apr 17 2023, 06:05 pm",applied,indeed.com,0
142,Data Scientist,PROFICIENTMANAGEMENT,"Apr 17 2023, 06:05 pm",applied,indeed.com,0


In [77]:
df = df_og

In [80]:
df['Category'] = 'Other'
df.loc[df['Title'].str.contains('Data Scientist', case=False), 'Category'] = 'Data Scientist'
df.loc[df['Title'].str.contains('analyst', case=False), 'Category'] = 'Data Analyst'
df.loc[df['Title'].str.contains('Data engineer', case=False), 'Category'] = 'Data Engineer'
df.loc[df['Title'].str.contains('Python', case=False), 'Category'] = 'Python Dev'
df.loc[df['Title'].str.contains('Machine Learning', case=False), 'Category'] = 'ML Engineer'

In [81]:
df

,Title,Company,Created at,List,Source,Ghosted after interview,Category
0,"Principal Data Scientist , Search and Personal...",Getty Images,"May 19 2023, 09:00 am",apply to,linkedin.com,0,Data Scientist
1,Trading Application Support Analyst,John Wiley & Sons,"May 19 2023, 08:22 am",apply to,linkedin.com,0,Data Analyst
2,Computational Cancer Biologist,Vevo Therapeutics,"May 19 2023, 07:00 am",applied,linkedin.com,0,Other
3,Data Engineer,Aja Hannah,"May 19 2023, 07:00 am",applied,linkedin.com,0,Data Engineer
4,Data Analytics Engineer/Specialist,ConocoPhillips,"May 19 2023, 06:39 am",applied,indeed.com,0,Other
...,...,...,...,...,...,...,...
139,Product Support Analyst,Central Michigan University,"Apr 17 2023, 06:05 pm",applied,indeed.com,0,Data Analyst
140,Performance Analyst – Hydroelectric energy,Innergex,"Apr 17 2023, 06:05 pm",applied,indeed.com,0,Data Analyst
141,"Data Analyst, Business Optimisation",VOSKER,"Apr 17 2023, 06:05 pm",applied,indeed.com,0,Data Analyst
142,Data Scientist,PROFICIENTMANAGEMENT,"Apr 17 2023, 06:05 pm",applied,indeed.com,0,Data Scientist


In [47]:


# clean up the sources 
source_list = ['indeed.com','linkedin.com','otta.com']
df['Source'] = df['Source'].apply(lambda x: x if x in source_list else 'other')

df = df[df['List'].str.contains('apply to') == False]


In [29]:
df['Source'].value_counts()

Source
indeed.com      68
linkedin.com    40
other           18
otta.com        16
Name: count, dtype: int64

In [30]:
df['List'].value_counts()

List
applied                     105
TRASH                        34
Rejected after interview      3
Name: count, dtype: int64

In [52]:
df['Ghosted after interview'].sum()

1

In [55]:
no_response_count

105

In [66]:
linkedin_count = df['Source'].value_counts()['linkedin.com']
indeed_count = df['Source'].value_counts()['indeed.com']
otta_count = df['Source'].value_counts()['otta.com']
other_count = df['Source'].value_counts()['other']
applied_count = df['List'].value_counts()['applied']
rejected_interview_count = df['List'].value_counts()['Rejected after interview'] - df['Ghosted after interview'].sum()
rejected_count =  df['List'].value_counts()['TRASH']
interview_count = df['List'].value_counts()['Rejected after interview']
ghosted_count = df['Ghosted after interview'].sum()
no_response_count = len(df)  - rejected_count - rejected_interview_count - ghosted_count


In [67]:
nodes = [
    {'label': 'Linkedin'},      # 0
    {'label': 'Indeed'},        # 1
    {'label': 'Otta'},          # 2 
    {'label': 'Other'},         # 3
    {'label': 'Applied'},       # 4
    {'label': 'No Response'},   # 5
    {'label': 'Rejected'},      # 6
    {'label': 'Ghosted'},       # 7
    {'label': 'Interview'},     # 8
]


# links = [
#     {'source': 'Linkedin', 'target': 'Applied', 'value': 10},  # Flow from Node 1 to Node 2 with value 10
#     {'source': 'Indeed', 'target': 'Applied', 'value': 5},
#     {'source': 'Otta', 'target': 'Applied', 'value': 5},   # Flow from Node 2 to Node 3 with value 5
#     {'source': 'Other', 'target': 'Applied', 'value': 5},
#     {'source': 'Applied', 'target': 'No Response', 'value': 5},
#     {'source': 'Applied', 'target': 'Rejected', 'value': 5},
#     {'source': 'Applied', 'target': 'Interview', 'value': 5},
#     {'source': 'Interview', 'target': 'Rejected', 'value': 5},
#     {'source': 'Applied', 'target': 'Ghosted', 'value': 5},

# ]

In [69]:
# Create the figure
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=[node['label'] for node in nodes],
    ),
    link = dict(
    source = [0,1,2,3,4,4,4,8,8],
    target = [4,4,4,4,5,6,8,7,6],
    value =  [linkedin_count,
               indeed_count,
               otta_count,
               other_count,
               no_response_count,
               rejected_count,
               interview_count,
               ghosted_count,
               rejected_interview_count
               ]
    )
    # link=dict(
    #     source=[link['source'] for link in links],
    #     target=[link['target'] for link in links],
    #     value=[link['value'] for link in links]
    # )
)])

# Display the figure
fig.show()